In [ ]:
%pip install window-slider

In [ ]:
import pandas as pd
import numpy as np
import csv
from scipy.stats import kurtosis,skew,iqr
from scipy import signal
from window_slider import Slider


In [ ]:
acc = '_acc'
gyro = '_gyro'
source = 'D:/UPES/UPES SEM VII/Major 1/endsem/HAPT Data Set/MyLab/'
dest = 'D:/UPES/UPES SEM VII/Major 1/endsem/HAPT Data Set/TimeSpectroSet/60-60/'
fs =50

In [ ]:
for i in range(1,13):
    accfile = source + str(i) + acc + '.csv'
    gyrofile = source + str(i) + gyro + '.csv'
    df1 = pd.read_csv(accfile)
    df2 = pd.read_csv(gyrofile)
    ax = df1['ax']
    ay = df1['ay']
    az = df1['az']
    wx = df2['wx']
    wy = df2['wy']
    wz = df2['wz']
    #------------------------------------------------------------------------------------------------------------------- 
    #Time domain
    timeAcc = np.empty([0,30])
    timeGyro = np.empty([0,30])

    size = 512
    overlap = 256

    
    for k in range(2):
        slider1 = Slider(size, overlap)
        slider2 = Slider(size, overlap)
        slider3 = Slider(size, overlap)
        if(k == 0):
            slider1.fit(ax)
            slider2.fit(ay)
            slider3.fit(az)
        elif(k == 1):
            slider1.fit(wx)
            slider2.fit(wy)
            slider3.fit(wz)        
        while True:
            window_data1 = slider1.slide()
            window_data2 = slider2.slide()
            window_data3 = slider3.slide()

            # do your stuff
            mean1 = np.mean(window_data1)
            variance1 = np.var(window_data1)
            median1 = np.median(window_data1)
            rms1 = np.sqrt(np.mean(window_data1**2))
            skewdness1 = skew(window_data1)
            kurtos1 = kurtosis(window_data1)
            ptp1 = np.ptp(window_data1)
            rang1 = np.max(window_data1) - np.min(window_data1)
            interq_range1 = iqr(window_data1)
            crest_factor1 = np.max(window_data1)/rms1

            mean2 = np.mean(window_data2)
            variance2 = np.var(window_data2)
            median2 = np.median(window_data2)
            rms2 = np.sqrt(np.mean(window_data2**2))
            skewdness2 = skew(window_data2)
            kurtos2 = kurtosis(window_data2)
            ptp2 = np.ptp(window_data2)
            rang2 = np.max(window_data2) - np.min(window_data2)
            interq_range2 = iqr(window_data2)
            crest_factor2 = np.max(window_data2)/rms2

            mean3 = np.mean(window_data3)
            variance3 = np.var(window_data3)
            median3 = np.median(window_data3)
            rms3 = np.sqrt(np.mean(window_data3**2))
            skewdness3 = skew(window_data3)
            kurtos3 = kurtosis(window_data3)
            ptp3 = np.ptp(window_data3)
            rang3 = np.max(window_data3) - np.min(window_data3)
            interq_range3 = iqr(window_data3)
            crest_factor3 = np.max(window_data3)/rms3

            time_dom = np.array([mean1,mean2,mean3,variance1,variance2,variance3,median1,median2,median3,
                                 rms1,rms2,rms3,skewdness1,skewdness2,skewdness3,kurtos1,kurtos2,kurtos3,
                                 ptp1,ptp2,ptp3,rang1,rang2,rang3,interq_range1,interq_range2,interq_range3, 
                                 crest_factor1,crest_factor2,crest_factor3])
            if( k == 0 ):
                timeAcc = np.append(timeAcc, [time_dom], axis=0)
            elif ( k == 1):
                timeGyro = np.append(timeGyro, [time_dom], axis=0)

            if slider1.reached_end_of_list(): break

#     print(timeAcc.shape)
#     print(timeGyro.shape)
            
        
    
    #Frequency Domain
    Afx, Atx, Ax = signal.spectrogram(ax, fs=fs,window=signal.get_window('hann',512),noverlap=256,nperseg=512)
    Afy, Aty, Ay = signal.spectrogram(ay, fs=fs,window=signal.get_window('hann',512),noverlap=256,nperseg=512)
    Afz, Atz, Az = signal.spectrogram(az, fs=fs,window=signal.get_window('hann',512),noverlap=256,nperseg=512)
    
    Wfx, Wtx, Wx = signal.spectrogram(wx, fs=fs,window=signal.get_window('hann',512),noverlap=256,nperseg=512)
    Wfy, Wty, Wy = signal.spectrogram(wy, fs=fs,window=signal.get_window('hann',512),noverlap=256,nperseg=512)
    Wfz, Wtz, Wz = signal.spectrogram(wz, fs=fs,window=signal.get_window('hann',512),noverlap=256,nperseg=512)
    
    Ax = Ax.T
    Ay = Ay.T
    Az = Az.T
    Wx = Wx.T
    Wy = Wy.T
    Wz = Wz.T
#     print(Ax.shape)
    resA = np.concatenate((Ax, Ay, Az), axis = 1)
    resW = np.concatenate((Wx, Wy, Wz), axis = 1)
#     print(resA.shape)
    
    for j in range(resA.shape[0]):
        sA = sorted(resA[j])
        sW = sorted(resW[j])
        featureAcc = sA[0:15] + sA[resA.shape[1]-15:resA.shape[1]]    #For 100 features set 50 to 25
        featureAcc.extend( timeAcc[j] )
        featureGyr = sW[0:15] + sW[resA.shape[1]-15:resA.shape[1]]    #For 100 features set 50 to 25
        featureGyr.extend( timeGyro[j] )
        final = np.concatenate((featureAcc,featureGyr))
        final = np.append( final, i )
#         print(final)
        with open (dest+str(i)+'_Time_SpectroFeatures.csv','a',newline='') as out_file:
            writer = csv.writer(out_file)
            writer.writerow(final)


In [ ]:
# from window_slider import Slider
# import numpy
# list = numpy.array([0, 1, 2, 3, 4, 5, 6, 7])
# bucket_size = 3
# overlap_count = 1
# slider = Slider(bucket_size,overlap_count)
# slider.fit(list)       
# while True:
#     window_data = slider.slide()
#     # do your stuff
#     print(window_data)
#     if slider.reached_end_of_list(): break